In [10]:
%reload_ext kedro.ipython

[05/07/25 16:56:46] INFO     Registered line magic '%reload_kedro'                                   ]8;id=135280;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=973210;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=191815;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=480749;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#65\65]8;;\

                    INFO     Resolved project path as:                                              ]8;id=46180;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=476307;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#180\180]8;;\
                             /Users/RoverHofman/Desktop/School/thesis/embedding-forecasting.                       
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/07/25 16:56:47] WARNING  /Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-pac ]8;id=270478;file:///Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=615590;file:///Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py#109\109]8;;\
                             kages/kedro/io/data_catalog.py:165: KedroDeprecationWarning:                          
                             `DataCatalog` has been deprecated and will be replaced by                             
                             `KedroDataCatalog`, in Kedro 1.0.0.Currently some `KedroDataCatalog`                  
                             APIs have been retained for compatibility with `DataCatalog`,                         
                             including the `datasets` property and the `get_datasets`,                             
                             `_get_datasets`, `add`,` list`, `add_feed_dict`, and `shallow_copy`                   
                             methods. These will be removed or replaced with updated alternatives                  
                             in Kedro 1.0.0. For more details, refer to the documentation:                         
                             https://docs.kedro.org/en/stable/data/index.html#kedrodatacatalog-expe                
                             rimental-feature                                                                      
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-pac ]8;id=59434;file:///Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=627669;file:///Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py#109\109]8;;\
                             kages/kedro/io/data_catalog.py:165: KedroDeprecationWarning:                          
                             `DataCatalog` has been deprecated and will be replaced by                             
                             `KedroDataCatalog`, in Kedro 1.0.0.Currently some `KedroDataCatalog`                  
                             APIs have been retained for compatibility with `DataCatalog`,                         
                             including the `datasets` property and the `get_datasets`,                             
                             `_get_datasets`, `add`,` list`, `add_feed_dict`, and `shallow_copy`                   
                             methods. These will be removed or replaced with updated alternatives                  
                             in Kedro 1.0.0. For more details, refer to the documentation:                         
                             https://docs.kedro.org/en/stable/data/index.html#kedrodatacatalog-expe                
                             rimental-feature                                                                      
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=660166;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=9255;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[05/07/25 16:56:48] INFO     Kedro project embedding_forecasting                                    ]8;id=198816;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=57167;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=848003;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=852211;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#147\147]8;;\
                             'pipelines'                                                                           

In [11]:
catalog.list()



['closing_prices', 'parameters']

In [12]:
catalog.load("closing_prices")

[05/07/25 16:56:53] INFO     Loading data from closing_prices (CSVDataset)...                   ]8;id=532659;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=901468;file:///Users/RoverHofman/Desktop/School/thesis/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#401\401]8;;\

,Date,price_close
0,1950-01-03,16.66
1,1950-01-04,16.85
2,1950-01-05,16.93
3,1950-01-06,16.98
4,1950-01-07,17.09
...,...,...
19033,2025-04-23,5375.86
19034,2025-04-24,5484.77
19035,2025-04-25,5525.21
19036,2025-04-28,5528.75
